<a href="https://colab.research.google.com/github/Thandeka20/High-spatial-resolution-imagery-vs-high-spectral-imagery/blob/main/Code/Aerial_Photograph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import ee
import folium
import time
import numpy as np
import pandas as pd
import geopandas as gpd
import re
import os
from IPython.display import Image
import geemap


In [12]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='mapwaps-sabiecroc')

In [13]:
# Define the image
image = ee.Image('projects/mapwaps-sabiecroc/assets/2530BB_24_2018_1321_RGB_RECT')

In [22]:
#visualize the RGB scene
# Define the bands to use for visualization
bands = ee.List(["b1", "b2", "b3"])

# Define visualization parameters
vis_params = {
    'bands': ['b1', 'b2', 'b3'],
    'min': 28,
    'max': 220,
}

# Example: Display an image using the defined visualization parameters
image = ee.Image('projects/mapwaps-sabiecroc/assets/2530BB_24_2018_1321_RGB_RECT')
display_image = Image(url=image.visualize(**vis_params).getThumbURL({'dimensions': '500x500'}))
display(display_image)

Extracting bands from aerial imagery


**Import image to GEE**
The first step is to import the image from GEE

In [15]:

def get_RGB_bands(image_id, latitude, longitude):
    """
    Function that extracts the spectral bands from a SPOT 7 satellite image of a specified location.

    Parameters:
        image_id (str): Unique Aerial image ID.
        latitude (float): Latitude coordinate of the desired location.
        longitude (float): Longitude coordinate of the desired location.

    Returns:
        band_dict (dict): Dictionary of multispectral band values for a specified location with labels b1-b3.
    """
    try:
        # Load the SPOT 7 image by its ID
        RGB_image = ee.Image(image_id)

        # Define a point geometry for the specified latitude and longitude
        point_geometry = ee.Geometry.Point([longitude, latitude])

        # Use the .sample() method to extract pixel values at the specified geometry
        # This will create a feature collection containing the pixel values
        pixel_values = RGB_image.sample(point_geometry, 2.5)  # Adjust scale as needed

        # Extract band values from the feature collection and add them to the dictionary
        band_dict = {}
        band_names = RGB_image.bandNames().getInfo()

        for band_name in band_names:
            band_value = pixel_values.first().get(band_name)
            band_dict[band_name] = ee.Number(band_value).getInfo()

        return band_dict

    except ee.EEException as e:
        return {"error": "An Earth Engine exception occurred: " + str(e)}
    except Exception as e:
        return {"error": "An unexpected error occurred: " + str(e)}

# -----------------------------------------------------------------------------------------------------------------
# Example usage:
RGB_image_id = 'projects/mapwaps-sabiecroc/assets/2530BB_24_2018_1321_RGB_RECT'
latitude = -25.22
longitude = 30.92

band_data = get_RGB_bands(RGB_image_id, latitude, longitude)
print(band_data)


{'b1': 48, 'b2': 64, 'b3': 61}


Adding indices

In [16]:
# Define vegetation indices
GI = image.expression('(GREEN)/(RED)', {
    'GREEN': image.select('b2'),
    'RED': image.select('b1'),
}).rename('GI')

IRG = image.expression('(RED-GREEN)', {
    'RED': image.select('b1'),
    'GREEN': image.select('b2'),
}).rename('IRG')

NGRDI = image.expression('(GREEN-RED)/(GREEN+RED)', {
    'RED': image.select('b1'),
    'GREEN': image.select('b2'),
}).rename('NGRDI')

VARI = image.expression('(GREEN-RED)/(GREEN+RED+BLUE)', {
    'RED': image.select('b1'),
    'GREEN': image.select('b2'),
    'BLUE': image.select('b3'),
}).rename('VARI')

VDVI = image.expression('(2*GREEN-RED-BLUE)/(2*GREEN+RED+BLUE)', {
    'RED': image.select('b1'),
    'GREEN': image.select('b2'),
    'BLUE': image.select('b3'),
}).rename('VDVI')

WBI = image.expression('(Blue - Red) / (Blue + Red)', {
    'Blue': image.select('b3'),
    'Red': image.select('b1'),
}).rename('WBI')

# Add computed variables as bands to the image_final
image_final = image.addBands([GI, IRG, NGRDI, VARI, VDVI, WBI])

# Print image bands
print(image_final.getInfo(), 'image_bands')

{'type': 'Image', 'bands': [{'id': 'b1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [21366, 23367], 'crs': 'PROJCS["Projection: Transverse Mercator; Datum: WGS84; Ellipsoid: WGS84", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH], \n    AUTHORITY["EPSG","4326"]], \n  PROJECTION["Transverse_Mercator"], \n  PARAMETER["central_meridian", 31.0], \n  PARAMETER["latitude_of_origin", 0.0], \n  PARAMETER["scale_factor", 1.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  UNIT["m", 1.0], \n  AXIS["Easting", EAST], \n  AXIS["Northing", NORTH]]', 'crs_transform': [0.25, 0, -10228.609567726324, 0, -0.25, -2788059.9809737173]}, {'id': 'b2', 'data_type': {'type': 'PixelType'

Import training data from GEE

In [19]:

# Load the training data (e.g., a FeatureCollection) from GEE
training_data = ee.FeatureCollection('projects/mapwaps-sabiecroc/assets/Training_setSc')

# Convert the training data to a GeoDataFrame
features = training_data.getInfo()['features']
training_subset = gpd.GeoDataFrame.from_features(features)

# Display the GeoDataFrame
print(training_subset.head())



                     geometry DIRECTION DIRECTIO_1      DateTime DateTime_1  \
0  POINT (30.83475 -25.53864)                      -2.209133e+12              
1  POINT (30.83556 -25.53828)                      -2.209133e+12              
2  POINT (30.94377 -25.06340)         E            -2.209133e+12              
3  POINT (30.93543 -25.06455)         E            -2.209133e+12              
4  POINT (30.97520 -24.95397)         E             1.691478e+12              

   Distance_1  Distance_t Done Done_1  FID_  ...    Val_id Val_id_1  \
0           0           0                 0  ...  Val_4440            
1           0           0                 0  ...  Val_4441            
2           0           5    Y            0  ...    Val_19            
3           0          30    Y            0  ...    Val_23            
4           0          15    Y            0  ...    Val_33            

   Validation          X          Y        Z  Z_1  coords_x1 coords_x2  \
0           0   0.000000

Run the classification using random forest

In [20]:
# Classification
classifier = ee.Classifier.smileRandomForest(100).train(
    features=training_data,
    classProperty='ID',
    inputProperties=bands
)

classified = image_final.select(bands).classify(classifier)

Define a colour palette

In [21]:
# Define the palette
SabieCrocPalette = [
    '351C75', 'F91DF9', '980A7D', '741b47', 'fd0618', 'E06666', 'ffcc99', 'ffffff', '999999', 'a8a800',
    '6aa84f', '14870e', 'DB992D', 'ff7f00', '000000', '0a14f9', '08f3e4'
]

# Clip the result to the region of interest
#LULC_Clipped = classified.clip(ROI)

# Visualization parameters
viz = {
    'min': 1,
    'max': 17,
    'palette': SabieCrocPalette
}

# Add the classified image to the map
#Map.addLayer(LULC_Clipped, viz, 'classification')

# Display the map
Map.centerObject(Point, 10)
Map.addLayer(ROI, {'color': 'FF0000'}, 'Point')
Map

NameError: name 'Map' is not defined